In [158]:
import numpy as np
import pandas as pd



def Ens(lee,K,S,nglt):
    '''
    Ens vector de ensamble 
    lee vector de colocacion. numerando los grados de libertad para cada elemento 
    K Matrices de Rigides de cada elemento en coordenadas de globales
    S Matriz de rigidez total de la estructura 
    nglt numero de grados de libertad incluido resticciones 
    '''

    ng = len(lee)
    for i in range (1,ng):                             # contador de filas 
        ii = lee[i]                                    # cuanta la posiscion ii en 1,1:2,2..etc
        if ii>0:
            if ii <= nglt:                             # restringe que que las posicio ii no supere los grados de libertad.
                for j in range (1,ng):
                    jj=lee[j]                          # cuanta la posiscion jj en 1,1:2,2..etc
                    if jj>0:
                        if jj <= nglt:                 # restringe que que las posicio jj no supere los grados de libertad.
                            S[ii,jj]=S[ii,jj]+ K[i,j]  # ensambla la matriz S con las coordenadas mas la matriz original K de cada miembro
    
    return S


                    


L = 1
A = (np.pi*(0.02**2))/4 #Area de cada elemento
 
E = 21000000  #Modulo de elasticidad del acero
AE = A*E
L1 = 2.1
L2 = 4.2
L3 = np.sqrt(L1**2 + L1**2)
L4 = np.sqrt(L2**2 + L2**2)
Long_el = [L3, L3, L2, L2, L1, L3, L1, L1, L1, L4, L2, L4, L2, L1, L1, L2, L2]            # longitudes de cada elemento
Ae_elem = [AE, AE, AE, AE, AE, AE, AE, AE, AE, AE, AE, AE, AE, AE, AE, AE, AE]            # E por A de cada elemento
nglt = 20                                                                                 # Grados de libertad incluidas restricciones 
gdl = 14                                                                                  # Grados de libertad 
primera_reaccion = 15                                                                     # Numeración de la primera reacción 

kmp = []
def k_elem_2x2(AE,L):
    kmp = [[AE/L, -AE/L], 
        [-AE/L, AE/L]]
    
    return kmp


def T_elem__4x2(alfas):
    Ta = [[np.cos(alfas), 0],
    [np.sin(alfas), 0],
    [0, np.cos(alfas)],
    [0, np.sin(alfas)]]

    return Ta


def Kelem_6GDL_MVA(E,I,G,L,A,f): #Nombre y parametros de la función

    #Declaracion de las constantes
    r = A*E/L
    B = 6*E*I*f/(G*A*L**2)
    kp = 2*E*I/L*(2+B)/(1+2*B)
    ap = 2*E*I/L*1-B/(1-2*B)
    bp = 6*E*I/L**(2)*1/(1+2*B)
    tp = 12*E*I/L**(3)*1/(1+2*B)
    #Declaracion de la matriz de rigidez
    k = [[r, 0, 0, -r, 0, 0]
        [0, tp, bp, 0, -tp, bp]
        [0, bp, kp, 0, -bp, ap]
        [-r, 0, 0, r, 0, 0]
        [0, -tp, -bp, 0, tp, -bp]
        [0, bp, ap, 0, -bp, kp]]
    
    return k


j = 1
k_ele_local = np.zeros((len(Long_el)*2,2))                     
for i in range (1, len(Long_el)):
    k_ele_local[j:j+2,:] = k_elem_2x2(Ae_elem[i],Long_el[i])
    j = j+2

k_ele_local_tbl = pd.DataFrame(k_ele_local)
print(k_ele_local_tbl)

angulo = [45, 45, 0, 0, 90, 135, 0, 90, 90, 135, 90, 135, 90, 0, 0, 0, 0]            # angulos de los elementos en grados

j=1
Ta = np.zeros((len(Long_el)*4,2))
for i in range (1,len(angulo)):
    Ta[j:j+4,:] = T_elem__4x2(angulo[i])
    j = j + 4

Ta_tbl = pd.DataFrame(Ta)
print(Ta_tbl)

Tat = np.transpose(Ta)
 
j=1
z=1
Ke_G = np.zeros((len(Long_el)*4,4))
for i in range (1,len(Long_el)):
    Ke_G[z:z+4,:] = Ta[z:z+4,:] @ k_ele_local[j-1:j+1,:] @ Tat[:,z:z+4]
    j=j+2
    z=z+4

Ke_G_tbl = pd.DataFrame(Ke_G)
print(Ke_G_tbl)


Lee = np.array( [[15, 16, 1, 2],   #Vector de ensamble elemento 1 
    [1, 2, 3, 4],     #Vector de ensamble elemento 2 
    [3, 4, 5, 6],     #Vector de ensamble elemento 3 
    [5, 6, 17, 18],   #Vector de ensamble elemento 4 
    [7, 8, 1, 2],    #Vector de ensamble elemento 5 
    [9, 19, 1, 2],    #Vector de ensamble elemento 6 
    [1, 2, 13, 14],   #Vector de ensamble elemento 7 
    [9, 19, 13, 14],  #Vector de ensamble elemento 8 
    [13, 14, 3, 4],   #Vector de ensamble elemento 9 
    [10, 11, 3, 4],  #Vector de ensamble elemento 10
    [10, 11, 5, 6],  #Vector de ensamble elemento 11
    [12, 20, 5, 6],  #Vector de ensamble elemento 12
    [12, 20, 17, 18],    #Vector de ensamble elemento 13
    [15, 16, 7, 8],  #Vector de ensamble elemento 14
    [7, 8, 9, 19],   #Vector de ensamble elemento 15
    [9, 19, 10, 11], #Vector de ensamble elemento 16
    [10, 11, 12, 20]])    #Vector de ensamble elemento 17


#Elms = ['Elemento1','Elemento2','Elemento3','Elemento4','Elemento5','Elemento6','Elemento7','Elemento8','Elemento9','Elemento10','Elemento11','Elemento12','Elemento13','Elemento14','Elemento15','Elemento16','Elemento17']
df = pd.DataFrame(Lee)
print(df)


S = np.zeros((nglt+1, nglt+1), dtype=int)     # Matriz de rigidez de la estructura (llena de ceros)
S_tbl = pd.DataFrame(S)
print(S_tbl)


j=1
for i in range (1,len(Long_el)):
    S = Ens(Lee[i,:], Ke_G[j:j+4,:], S, nglt)
    j = j + 4

#GDLs = ['D1','D2','D3','D4','D5','D6','D7','D8','D9','D10','D11','D12','D13','D14','R15','R16','R17','R18','R19','R20']
Tabla = pd.DataFrame(S)
print(Tabla)


Sll = S[1:gdl,1:gdl]
#GDLs = ['D1','D2','D3','D4','D5','D6','D7','D8','D9','D10','D11','D12','D13','D14']
Sll_tbl = pd.DataFrame(Sll)

P = [0,2.5,1,0,0,0,0,0,0,0,0,0,0]

Delta_g = np.zeros(nglt+1)
Delta_g[0:len(np.dot(np.linalg.inv(Sll), np.array(P)))] = np.dot(np.linalg.inv(Sll), np.array(P))
#GDLs = ['D1','D2','D3','D4','D5','D6','D7','D8','D9','D10','D11','D12','D13','D14']

#GDLs = ['D1','D2','D3','D4','D5','D6','D7','D8','D9','D10','D11','D12','D13','D14','R15','R16','R17','R18','R19','R20']
Delta_g_tbl = pd.DataFrame(Delta_g)
print(Delta_g_tbl)

j = 1
Delta_g_ij = np.zeros((len(Long_el)*4))
for i in range(1, len(Long_el)):
    Delta_g_ij[j:j+4] = Delta_g[np.transpose(Lee[i,:])]
    j = j + 4

Tabla = pd.DataFrame({'Delta_g_ij': Delta_g_ij})
print(Tabla)



j=1
z=1
Delta_L_ij = np.zeros(len(Long_el)*2)
for i in range(1, len(Long_el)):
    Delta_L_ij[z:z+2] = Tat[:,j:j+4] @ Delta_g_ij[j:j+4]
    j += 4
    z += 2

Delta_L_ij_tbl = pd.DataFrame(Delta_L_ij)
print(Delta_L_ij_tbl)


z=1
acc_elem_L = np.zeros(len(Long_el)*2)
for i in range(1, len(Long_el)):
    acc_elem_L[z:z+2] = k_ele_local[z:z+2,:] @ Delta_L_ij[z:z+2]
    z += 2

acc_elem_L_tbl = pd.DataFrame(acc_elem_L)
#FXs = ['FXa1','FXb1','FXa2','FXb2','FXa3','FXb3','FXa4','FXb4','FXa5','FXb5','FXa6','FXb6','FXa7','FXb7','FXa8','FXb8','FXa9','FXb9','FXa10','FXb10','FXa11','FXb11','FXa12','FXb12','FXa13','FXb13','FXa14','FXb14','FXa15','FXb15','FXa16','FXb16','FXa17','FXb17']
pd.DataFrame(acc_elem_L_tbl)




              0            1
0      0.000000     0.000000
1   2221.441469 -2221.441469
2  -2221.441469  2221.441469
3   1570.796327 -1570.796327
4  -1570.796327  1570.796327
5   1570.796327 -1570.796327
6  -1570.796327  1570.796327
7   3141.592654 -3141.592654
8  -3141.592654  3141.592654
9   2221.441469 -2221.441469
10 -2221.441469  2221.441469
11  3141.592654 -3141.592654
12 -3141.592654  3141.592654
13  3141.592654 -3141.592654
14 -3141.592654  3141.592654
15  3141.592654 -3141.592654
16 -3141.592654  3141.592654
17  1110.720735 -1110.720735
18 -1110.720735  1110.720735
19  1570.796327 -1570.796327
20 -1570.796327  1570.796327
21  1110.720735 -1110.720735
22 -1110.720735  1110.720735
23  1570.796327 -1570.796327
24 -1570.796327  1570.796327
25  3141.592654 -3141.592654
26 -3141.592654  3141.592654
27  3141.592654 -3141.592654
28 -3141.592654  3141.592654
29  1570.796327 -1570.796327
30 -1570.796327  1570.796327
31  1570.796327 -1570.796327
32 -1570.796327  1570.796327
33     0.00000

<>:74: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
<>:74: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
C:\Users\USUARIO\AppData\Local\Temp\ipykernel_8872\4243252042.py:74: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
  k = [[r, 0, 0, -r, 0, 0]


,0
0,0.000000
1,26600.633524
2,-26600.633524
3,129453.600001
4,-129453.600001
5,-122622.865600
6,122622.865600
7,-21794.732696
8,21794.732696
9,19987.033204
